In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import requests
import xmltodict

class AptTradingDataloader(object):
    def __init__(self):
        self.authKey = 'G7GuVfz99A0umUDOAuZlRXMYlZmKxBd3hHsj8EiZ89SSD7r5qrn08q70LM96fyblxiM3eGxWsgkIjhF%2BAWTVBg%3D%3D'
        self.raw_cols = {"aptDong": "단지동명", "aptNm": "단지명", "aptSeq": "단지일련번호", "bonbun": "법정동본번", "bubun": "법정동부번",
                     "buildYear": "건축년도", "buyerGbn": "매수자유형명", "cdealDay": "해제사유발생일자", "cdealType": "해제여부",
                     "dealAmount": "거래금액", "dealDay": "거래일자", "dealMonth": "거래월", "dealYear": "거래년도", "dealingGbn": "거래구분명",
                     "estateAgentSggNm": "중개사소재시군구명", "excluUseAr": "전용면적제곱미터", "floor": "층수", "jibun": "법정동지번내용",
                     "landCd": "대지구분코드", "landLeaseholdGbn": "토지임대여부", "rgstDate": "등기일자", "roadNm": "도로명",
                     "roadNmBonbun": "도로명본번", "roadNmBubun": "도로명부번", "roadNmCd": "도로명코드", "roadNmSeq": "도로명일련번호",
                     "roadNmSggCd": "도로명시군구코드", "roadNmbCd": "도로명지상지하코드", "sggCd": "법정동시군구코드", "slerGbn": "매도자유형명",
                     "umdCd": "법정동읍면동코드", "umdNm": "법정동읍면동명"}
        self.data_cols = ['거래년도', '거래월', '거래일자', '등기일자', ## 언제
                          '법정동시군구코드', '법정동읍면동코드', '법정동읍면동명', '대지구분코드', '법정동본번', '법정동부번', '법정동지번내용', ## 어디서
                          '도로명시군구코드', '도로명코드', '도로명일련번호', '도로명본번', '도로명부번', '도로명지상지하코드', '도로명', ## 어디서
                          '단지일련번호', '단지명', '단지동명', '건축년도', '전용면적제곱미터', '층수', '토지임대여부', ## 무엇을
                          '거래구분명', '거래금액', '매수자유형명', '매도자유형명', ## 어떻게
                          '해제여부', '해제사유발생일자', '중개사소재시군구명'] ## 기타
        self.cols = ['거래년도', '거래월', '거래일자', ## 언제
                     '법정동시군구코드', '법정동읍면동코드', ## 어디서
                     '단지일련번호', '단지명', '전용면적제곱미터', '층수', '건축년도', ## 무엇을
                     '거래구분명', '거래금액', ## 어떻게
                     '해제여부'] ## 기타

    ## APT 매매이력 API
    def get_apt_trading_raw(self, sigungu_cd, base_month):
        authKey = self.authKey
        try:
            num_of_rows = 100
            url = f"http://apis.data.go.kr/1613000/RTMSDataSvcAptTradeDev/getRTMSDataSvcAptTradeDev?serviceKey={authKey}&LAWD_CD={sigungu_cd}&DEAL_YMD={base_month}&pageNo=1&numOfRows={num_of_rows}"
            response = requests.get(url)
            raw = xmltodict.parse(response.text)
            total_cnt = int(raw['response']['body']['totalCount'])
            max_page = total_cnt//num_of_rows + 1
            raw = raw['response']['body']['items']['item']
            dataset = pd.DataFrame(raw)
            if max_page > 1:
                for page_no in range(2, max_page+1):
                    url = f"http://apis.data.go.kr/1613000/RTMSDataSvcAptTradeDev/getRTMSDataSvcAptTradeDev?serviceKey={authKey}&LAWD_CD={sigungu_cd}&DEAL_YMD={base_month}&pageNo={page_no}&numOfRows={num_of_rows}"
                    response = requests.get(url)
                    raw = xmltodict.parse(response.text)
                    raw = raw['response']['body']['items']['item']
                    dataset = pd.concat([dataset, pd.DataFrame(raw)])
            return dataset
        except Exception as e:
            return print(f'{base_month}, {sigungu_cd} 결과를 가져오는데 실패했습니다.')
        
    def preproc_apt_trading(self, apt_trading_raw):
        raw_cols = self.raw_cols
        data_cols = self.data_cols
        cols = self.cols
        dataset = apt_trading_raw.rename(columns=raw_cols)
        dataset = dataset[data_cols]
        df = dataset[cols]
        return dataset, df

In [12]:
module = AptTradingDataloader()

In [13]:
raw_df = module.get_apt_trading_raw('11215', '202410')
dataset, df = module.preproc_apt_trading(raw_df)

In [14]:
df

,거래년도,거래월,거래일자,법정동시군구코드,법정동읍면동코드,단지일련번호,단지명,전용면적제곱미터,층수,건축년도,거래구분명,거래금액,해제여부
0,2024,10,19,11215,10400,11215-4,극동2,102.61,3,1989,중개거래,"208,000",None
1,2024,10,7,11215,10400,11215-2,광장현대9단지,59.97,21,1999,중개거래,"122,000",None
2,2024,10,11,11215,10500,11215-51,대동,84.65,3,1999,중개거래,"120,500",None
3,2024,10,16,11215,10400,11215-16,현대3,74.92,10,1990,중개거래,"138,500",None
4,2024,10,12,11215,10400,11215-4,극동2,75.55,8,1989,중개거래,"171,000",None
5,2024,10,18,11215,10400,11215-2919,유진스웰,165.327,7,2011,직거래,"190,000",None
6,2024,10,19,11215,10500,11215-74,자양7차현대홈타운,59.94,8,2001,중개거래,"117,000",None
7,2024,10,12,11215,10500,11215-56,우성2,80.1,5,1989,중개거래,"112,500",None
8,2024,10,2,11215,10500,11215-72,한양,166.93,1,1983,중개거래,"210,000",None
9,2024,10,12,11215,10400,11215-144,상록타워아파트,84.9,5,1997,중개거래,"120,000",None


In [76]:
dataset = get_apt_trading("11215", "202410", authKey)
dataset = dataset.rename(columns=cols)

In [77]:
dataset.columns

Index(['단지동명', '단지명', '단지일련번호', '법정동본번', '법정동부번', '건축년도', '매수자유형명', '해제사유발생일자',
       '해제여부', '거래금액', '거래일자', '거래월', '거래년도', '거래구분명', '중개사소재시군구명', '전용면적제곱미터',
       '층수', '법정동지번내용', '대지구분코드', '토지임대여부', '등기일자', '도로명', '도로명본번', '도로명부번',
       '도로명코드', '도로명일련번호', '도로명시군구코드', '도로명지상지하코드', '법정동시군구코드', '매도자유형명',
       '법정동읍면동코드', '법정동읍면동명'],
      dtype='object')

In [53]:
dataset[['거래년도', '거래월', '거래일자']]

SyntaxError: invalid syntax (2349148353.py, line 2)

,법정동시군구코드,법정동읍면동코드,법정동읍면동명,대지구분코드,법정동본번,법정동부번,법정동지번내용
0,11215,10400,광장동,1,0218,0001,218-1
1,11215,10400,광장동,1,0576,0000,576
2,11215,10500,자양동,1,0784,0001,784-1
3,11215,10400,광장동,1,0484,0000,484
4,11215,10400,광장동,1,0218,0001,218-1
5,11215,10400,광장동,1,0121,0000,121
6,11215,10500,자양동,1,0788,0000,788
7,11215,10500,자양동,1,0520,0002,520-2
8,11215,10500,자양동,1,0695,0000,695
9,11215,10400,광장동,1,0570,0000,570


In [40]:
[, , , '해제사유발생일자',
       , , , '중개사소재시군구명', ,
       , , , , , ]

,aptDong,aptNm,aptSeq,bonbun,bubun,buildYear,buyerGbn,cdealDay,cdealType,dealAmount,...,roadNmBonbun,roadNmBubun,roadNmCd,roadNmSeq,roadNmSggCd,roadNmbCd,sggCd,slerGbn,umdCd,umdNm
0,None,극동2,11215-4,0218,0001,1989,개인,None,None,"208,000",...,00552,00000,3000002,02,11215,0,11215,개인,10400,광장동
1,None,광장현대9단지,11215-2,0576,0000,1999,개인,None,None,"122,000",...,00019,00000,4112352,00,11215,NaN,11215,개인,10400,광장동
2,None,대동,11215-51,0784,0001,1999,개인,None,None,"120,500",...,00020,00000,4112253,01,11215,0,11215,개인,10500,자양동
3,None,현대3,11215-16,0484,0000,1990,개인,None,None,"138,500",...,00062,00000,4112353,01,11215,0,11215,개인,10400,광장동
4,None,극동2,11215-4,0218,0001,1989,개인,None,None,"171,000",...,00552,00000,3000002,02,11215,0,11215,개인,10400,광장동
5,None,유진스웰,11215-2919,0121,0000,2011,개인,None,None,"190,000",...,00007,00000,4112359,01,11215,0,11215,개인,10400,광장동
6,None,자양7차현대홈타운,11215-74,0788,0000,2001,개인,None,None,"117,000",...,00041,00000,4112166,01,11215,0,11215,개인,10500,자양동
7,None,우성2,11215-56,0520,0002,1989,개인,None,None,"112,500",...,00576,00000,3103010,02,11215,0,11215,개인,10500,자양동
8,None,한양,11215-72,0695,0000,1983,개인,None,None,"210,000",...,00011,00000,3104003,02,11215,0,11215,개인,10500,자양동
9,None,상록타워아파트,11215-144,0570,0000,1997,개인,None,None,"120,000",...,00540,00000,3000002,02,11215,0,11215,개인,10400,광장동


In [36]:
//num_of_rows

0

In [33]:
raw['response']['body']['totalCount']//num_of_rows + 1

'29'

In [27]:

raw = raw['response']['body']['items']['item']

,aptDong,aptNm,aptSeq,bonbun,bubun,buildYear,buyerGbn,cdealDay,cdealType,dealAmount,...,roadNmBonbun,roadNmBubun,roadNmCd,roadNmSeq,roadNmSggCd,roadNmbCd,sggCd,slerGbn,umdCd,umdNm
0,None,극동2,11215-4,0218,0001,1989,개인,None,None,"208,000",...,00552,00000,3000002,02,11215,0,11215,개인,10400,광장동
1,None,광장현대9단지,11215-2,0576,0000,1999,개인,None,None,"122,000",...,00019,00000,4112352,00,11215,NaN,11215,개인,10400,광장동
2,None,대동,11215-51,0784,0001,1999,개인,None,None,"120,500",...,00020,00000,4112253,01,11215,0,11215,개인,10500,자양동
3,None,현대3,11215-16,0484,0000,1990,개인,None,None,"138,500",...,00062,00000,4112353,01,11215,0,11215,개인,10400,광장동
4,None,극동2,11215-4,0218,0001,1989,개인,None,None,"171,000",...,00552,00000,3000002,02,11215,0,11215,개인,10400,광장동
5,None,유진스웰,11215-2919,0121,0000,2011,개인,None,None,"190,000",...,00007,00000,4112359,01,11215,0,11215,개인,10400,광장동
6,None,자양7차현대홈타운,11215-74,0788,0000,2001,개인,None,None,"117,000",...,00041,00000,4112166,01,11215,0,11215,개인,10500,자양동
7,None,우성2,11215-56,0520,0002,1989,개인,None,None,"112,500",...,00576,00000,3103010,02,11215,0,11215,개인,10500,자양동
8,None,한양,11215-72,0695,0000,1983,개인,None,None,"210,000",...,00011,00000,3104003,02,11215,0,11215,개인,10500,자양동
9,None,상록타워아파트,11215-144,0570,0000,1997,개인,None,None,"120,000",...,00540,00000,3000002,02,11215,0,11215,개인,10400,광장동


In [2]:
authKey = 'G7GuVfz99A0umUDOAuZlRXMYlZmKxBd3hHsj8EiZ89SSD7r5qrn08q70LM96fyblxiM3eGxWsgkIjhF%2BAWTVBg%3D%3D'

## APT 매매이력 API
def get_apt_trading(sigungu_cd, base_month, authKey = authKey):
    url = f"http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTradeDev?serviceKey={authKey}&pageNo=1&numOfRows=100000&LAWD_CD={sigungu_cd}&DEAL_YMD={base_month}"
    try:
        response = requests.get(url)
        raw = xmltodict.parse(response.text)
        raw = raw['response']['body']['items']['item']
        return pd.DataFrame(raw)
    except Exception as e:
        return print(f'기준년월: {base_month}, 시군구코드: {sigungu_cd} 결과를 가져오는데 실패했습니다.')

In [3]:
## 법정동코드 불러오기
dl_path = 'C:/Users/GCC/git/data/DL/'
bjd_cd = pd.read_csv(f'{dl_path}법정동코드 전체자료.txt', encoding='cp949', sep='\t', dtype=str)
bjd_cd['시군구코드'] = bjd_cd['법정동코드'].str.slice(0, 5)
bjd_cd['시도코드'] = bjd_cd['법정동코드'].str.slice(0, 2)

In [4]:
## 조회조건 정의
sigungu_cd_ls = (bjd_cd[((bjd_cd['시도코드'] + '000') != bjd_cd['시군구코드'])&(bjd_cd['폐지여부'] == '존재')&
                        ((bjd_cd['시도코드'] == '11')|(bjd_cd['시도코드'] == '41'))]['시군구코드']).unique()
base_month_ls = pd.date_range('2006-01-01', '2024-01-31', freq='M').strftime('%Y%m').tolist()

In [5]:

import concurrent.futures
_ls = []
with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
    for sigungu_cd in sigungu_cd_ls:
        print(f'{sigungu_cd} 시작')
        future_month = {executor.submit(get_apt_trading, sigungu_cd, month): month for month in base_month_ls}
        for future in concurrent.futures.as_completed(future_month):
            dataset = future.result()
            _ls.append(dataset)
        print(f"done {sigungu_cd}")
trading_df = pd.concat(_ls)

11110 시작
done 11110
11140 시작
done 11140
11170 시작
done 11170
11200 시작
done 11200
11215 시작
done 11215
11230 시작
done 11230
11260 시작
done 11260
11290 시작
done 11290
11305 시작
done 11305
11320 시작
done 11320
11350 시작
done 11350
11380 시작
done 11380
11410 시작
done 11410
11440 시작
done 11440
11470 시작
done 11470
11500 시작
done 11500
11530 시작
done 11530
11545 시작
done 11545
11560 시작
done 11560
11590 시작
done 11590
11620 시작
done 11620
11650 시작
done 11650
11680 시작
done 11680
11710 시작
done 11710
11740 시작
done 11740
41110 시작
기준년월: 200601, 시군구코드: 41110 결과를 가져오는데 실패했습니다.
기준년월: 200702, 시군구코드: 41110 결과를 가져오는데 실패했습니다.
기준년월: 200602, 시군구코드: 41110 결과를 가져오는데 실패했습니다.
기준년월: 200611, 시군구코드: 41110 결과를 가져오는데 실패했습니다.
기준년월: 200605, 시군구코드: 41110 결과를 가져오는데 실패했습니다.
기준년월: 200609, 시군구코드: 41110 결과를 가져오는데 실패했습니다.
기준년월: 200606, 시군구코드: 41110 결과를 가져오는데 실패했습니다.
기준년월: 200705, 시군구코드: 41110 결과를 가져오는데 실패했습니다.
기준년월: 200703, 시군구코드: 41110 결과를 가져오는데 실패했습니다.
기준년월: 200603, 시군구코드: 41110 결과를 가져오는데 실패했습니다.
기준년월: 200612, 시군구코드: 41110 결과를 가져오는데 실패했습

In [7]:
dl_path

'C:/Users/GCC/git/data/DL/'

In [8]:
trading_df.to_csv(f'{dl_path}/매매내역_서울경기.csv', index=False, encoding='utf-8-sig')